In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.mam.*
import jline.solvers.nc.*
import jline.util.matrix.*

In [ ]:
// This model exemplifies how to specify models with multiple sinks (virtual sinks)
val model = Network("model")

// Define network nodes
val source = Source(model, "Source")
val queue = Queue(model, "Queue1", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")
val vsink1 = Router(model, "VSink1")  // Virtual sink 1
val vsink2 = Router(model, "VSink2")  // Virtual sink 2

// Define job classes
val ocl1 = OpenClass(model, "Class1")
val ocl2 = OpenClass(model, "Class2")

In [ ]:
// Set arrival processes
source.setArrival(ocl1, Exp(1.0))  // Class1: arrival rate 1.0
source.setArrival(ocl2, Exp(1.0))  // Class2: arrival rate 1.0

In [ ]:
// Set service processes
queue.setService(ocl1, Exp(100.0))  // Class1: very fast service (rate 100)
queue.setService(ocl2, Exp(100.0))  // Class2: very fast service (rate 100)

In [ ]:
// Configure routing matrix with virtual sinks
val P = model.initRoutingMatrix()

// Class1 routing
P.set(ocl1, ocl1, source, queue, 1.0)     // Source → Queue1
P.set(ocl1, ocl1, queue, vsink1, 0.6)     // Queue1 → VSink1 (60%)
P.set(ocl1, ocl1, queue, vsink2, 0.4)     // Queue1 → VSink2 (40%)
P.set(ocl1, ocl1, vsink1, sink, 1.0)      // VSink1 → Sink
P.set(ocl1, ocl1, vsink2, sink, 1.0)      // VSink2 → Sink

// Class2 routing
P.set(ocl2, ocl2, source, queue, 1.0)     // Source → Queue1
P.set(ocl2, ocl2, queue, vsink1, 0.1)     // Queue1 → VSink1 (10%)
P.set(ocl2, ocl2, queue, vsink2, 0.9)     // Queue1 → VSink2 (90%)
P.set(ocl2, ocl2, vsink1, sink, 1.0)      // VSink1 → Sink
P.set(ocl2, ocl2, vsink2, sink, 1.0)      // VSink2 → Sink

model.link(P)

In [ ]:
// Solve with MVA solver
println("\n=== MVA Solver ===")
val solverMva = MVA(model)
val avgTable = solverMva.avgTable
avgTable.print()

println("\n=== MVA Node Table ===")
val avgNodeTable = solverMva.avgNodeTable
avgNodeTable.print()

In [ ]:
// We use getAvgNodeTable to see the throughputs of sink1 and sink2
println("=== MVA Solver ===")
val avgTable = MVA(model).avgTable
avgTable.print()

val avgNodeTable = MVA(model).avgNodeTable
avgNodeTable.print()

In [ ]:
println("\n=== MAM Solver ===")
val avgTableMAM = MAM(model).avgTable
avgTableMAM.print()

val avgNodeTableMAM = MAM(model).avgNodeTable
avgNodeTableMAM.print()

println("\n=== NC Solver ===")
val avgTableNC = NC(model).avgTable
avgTableNC.print()

val avgNodeTableNC = NC(model).avgNodeTable
avgNodeTableNC.print()